In [33]:
Packages <- c("dplyr", "R.utils", "nleqslv", "broom","cubature", "geosphere", "data.table",  "ggplot2", "bbmle", "stringr",  "lubridate", "RColorBrewer")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
#source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_bbmle.R")
#source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum0.5.R") #integrate_kernel_sum1
#source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum1.R") #integrate_kernel_sum0.5
#source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve.R") #median
#source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve90.R") #dist 90% retained

#read in the kernel fitting summary
kernels <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/kernel_fitting_summary.csv")
#read in the centroids adjusted for the simulation, so the Magbangons combined 
#centroids <- fread(file="~/oceanography/script_output/SurveyData/SimulationCentroids.csv")
#read in the table with number of recruits sampled at each site for each year
AnnualRecsSamp <- fread(file="~/oceanography/script_output/SurveyData/AnnualRecruitsSampled.csv")
#read in the table of the proportion of anemones sampled at each site for each year
PropSamp <- fread(file="~/oceanography/script_output/SurveyData/ProportionHabitatSampled.csv")

#read in the ROMS simulation connectivity table
#SimConn <- fread(file="~/oceanography/script_output/ROMSDataTables/SimConnectivityTable.csv.gz")
#read in the ROMS simulation connectivity table with metadata, not yet subsetted (*but check this)
date_join <- fread(file="~/oceanography/script_output/LongFormConnWithProbs.csv")
#make vectors defining sites we didn't sample, but that are in the model, and the sandflats specifically 
UnsampledSites <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6", "Pangasugan", "Other", "CAI") 
SandFlats <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6") 

##make the constant inputs for the kernel fitting function
##distance matrix using the centroids with combined Magbangon
#### List of source locations
#sites_source <- centroids
#
#### List of destination locations
#sites_dest <- centroids
#
#dist_mat_m <- distm(sites_source[,c('lon','lat')], sites_source[,c('lon','lat')], fun=distVincentyEllipsoid)
#Distances <- dist_mat_m*10^-3
##read in the reef areas for the kernel fitting
#Area <- read.csv("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels.csv", header=TRUE, stringsAsFactors = F) %>%
#    arrange(site) %>%
#    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
#    mutate(kmsq=msq*10^-6) %>%
#    select(kmsq)
#Reef_sizes <- as.matrix(Area)

#make a table with the survey information for each site (how many fish sampled, prop anems sampled, total number of anems at site)
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year=end_year, site)][#join the sampling tables together
    is.na(n_offs_gen), n_offs_gen := 0][#change NA's to 0
    ,-"time_frame"]#drop the time_frame column, we can key with end_year
#setkey(SurveyData, site)

__Loop through sampling different proportions of other and CAI source particles and compare the unassigned proportions of total sample particles to the genetic observations from survey data__

In [16]:
PropAssignedTable <- rbind(kernels[Year %in% c("2012", "2013", "2014")][
    , PropAssigned := PercentAssigned/100][ #change to proportion note percent
    , c("Year", "PropAssigned")],                      
    unique(kernels[Year %in% c("2012", "2013", "2014")][ #only the years coinciding with the models
   , PropAssigned := sum(NumParentageMatches)/sum(NumOffsSampled)][#sum up all of the sampled offspring 2012-2014, and matches
    , "PropAssigned"][ #select the column with the total value
    , Year := "2012-14"]))

PropAssignedTable[]

Year,PropAssigned
2012,0.04760000
2013,0.14000000
2014,0.07180000
2012-14,0.09390863


In [34]:
head(date_join)

YearSampled,date,SurvWeight,Dest,DestNOffsAnnual,DestPropSamp,DestTotalAnems,Source,SourceNOffsAnnual,SourcePropSamp,SourceTotalAnems,SimMonth,SimDay,SimYear,SimMonsoon,DailyParticles,ParticleID,SourceSize
2011,2010-10-01,0.004115226,Palanas,NA,NA,NA,Other,NA,NA,NA,10,10,2010,SWM,9,P1,NA
2011,2010-10-01,0.004115226,Palanas,NA,NA,NA,Other,NA,NA,NA,10,10,2010,SWM,9,P2,NA
2011,2010-10-01,0.004115226,Palanas,NA,NA,NA,Other,NA,NA,NA,10,10,2010,SWM,9,P3,NA
2011,2010-10-01,0.004115226,Palanas,NA,NA,NA,Other,NA,NA,NA,10,10,2010,SWM,9,P4,NA
2011,2010-10-01,0.004115226,Palanas,NA,NA,NA,Other,NA,NA,NA,10,10,2010,SWM,9,P5,NA
2011,2010-10-01,0.004115226,Palanas,NA,NA,NA,Other,NA,NA,NA,10,10,2010,SWM,9,P6,NA


In [21]:
###outside of the loop
PropToEval <- seq(0.1, 1, 0.05) #make a vector of proportions to sample iterativaley and compare
#empty table to hold results
PropSampOtherCAI <- data.table(TimeScale=character(), TimeID=character(), PropAssigned=numeric(), PropSampEval=numeric(), Check1=character(), Check2=character())


In [38]:
#for(i in 1:length(PropToEval)){
i=0.1
PropSampOtherCAI_int <- data.table(TimeScale=character(), TimeID=character(), PropAssigned=numeric(), PropSampEval=numeric(), Check1=character(), Check2=character())

dest_sampled <- date_join[DestPropSamp >0]
check1 <- nrow(dest_sampled)
dest_sampled <- dest_sampled[c(dest_sampled[, .I[Source != "Other"]], sample(dest_sampled[, .I[Source == "Other"]], length(dest_sampled[, .I[Source == "Other"]])*i))]
check2 <- nrow(dest_sampled)
dest_sampled <- dest_sampled[c(dest_sampled[, .I[Source != "CAI"]], sample(dest_sampled[, .I[Source == "CAI"]], length(dest_sampled[, .I[Source == "CAI"]])*i))]
check3 <- nrow(dest_sampled)

#check that we have less rows, should both be TRUE
check1 > check2
check2 > check3

#join in the number of parentage matches observed by year
dest_sampled <- kernels[Year %in% c("2012", "2013", "2014")][, Year:=as.integer(Year)][,c("Year", "NumParentageMatches")][dest_sampled, on=.(Year=YearSampled)]#[

#randomly subsample the sampled particle data
sim_sample <- dest_sampled[, .SD[sample(.N, DestNOffsAnnual, prob=SurvWeight)], by = c("Year", "Dest")] #randomly sample rows (particles) from the table according to the survival weighting, based on the number we sampled at each site in each year of surveys
#check1 <- nrow(sim_sample)

#assign parentage
sim_parentage <- sim_sample[SourcePropSamp > 0][, .SD[sample(.N, NumParentageMatches, prob=SourcePropSamp)], by = .(Year)][#now randomly assign parentage or not parentage, based on how well we sampled the source and the number of parentage matches we had in that year
                , Parentage := 1]
#for faster searching, set keys
setkey(sim_parentage, ParticleID)
setkey(sim_sample, ParticleID)

l <- list(sim_sample[ParticleID %!in% sim_parentage$ParticleID][, Parentage := 0], sim_parentage)
sim_sample <- rbindlist(l, use.names = TRUE, fill=TRUE, idcol = NULL)[, c("Year", "Source", "Dest", "Parentage")] #add back in to the unassigned particles, select only the columns necessary

#}

[1] TRUE

[1] TRUE

In [40]:
(sim_sample)

Year,Source,Dest,Parentage
2012,Other,Wangag,0
2012,Other,Wangag,0
2013,Other,Haina,0
2013,Caridad Cemetery,Palanas,0
2013,Magbangon,Palanas,0
2013,Other,Elementary School,0
2013,Other,Wangag,0
2013,Other,Wangag,0
2013,Poroc San Flower,Palanas,0
2013,Poroc San Flower,Wangag,0


In [26]:
PropSampOtherCAI <- data.table(TimeScale=character(), TimeID=character(), PropAssigned=numeric(), PropSampEval=numeric(), Check1=character(), Check2=character())
PropSampOtherCAI

TimeScale,TimeID,PropAssigned,PropSampEval,Check1,Check2


In [6]:

dest_sampled <- date_join[DestPropSamp >0]
check1 <- nrow(dest_sampled)
dest_sampled <- dest_sampled[c(dest_sampled[, .I[Source != "Other"]], sample(dest_sampled[, .I[Source == "Other"]], length(dest_sampled[, .I[Source == "Other"]])*.1))]
check2 <- nrow(dest_sampled)
dest_sampled <- dest_sampled[c(dest_sampled[, .I[Source != "CAI"]], sample(dest_sampled[, .I[Source == "CAI"]], length(dest_sampled[, .I[Source == "CAI"]])*.1))]
check3 <- nrow(dest_sampled)

#check that we have less rows, should both be TRUE
check1 > check2
check2 > check3

#join in the number of parentage matches observed by year
dest_sampled <- kernels[Year %in% c("2012", "2013", "2014")][, Year:=as.integer(Year)][,c("Year", "NumParentageMatches")][dest_sampled, on=.(Year=YearSampled)]#[
    #, ParticleID := paste ("P", .I, sep = "", collapse = NULL)]#finally, make a new column that's the particle ID for each row

#setkey(dest_sampled, "Year", "Dest", "Source")




[1] TRUE

[1] TRUE

[1] TRUE

In [10]:
nrow(dest_sampled)
nrow(SimConn)

[1] 363760

[1] 212861852

In [7]:
#save inter file
#fwrite(dest_sampled, file="~/oceanography/script_output/LongFormConnWithProbsTest.csv")
#see if I can write as a compressed file so it can be stored on github
#https://stackoverflow.com/questions/42788401/is-possible-to-use-fwrite-from-data-table-with-gzfile